In [1]:
import pandas as pd 
import urllib.request, json
#import requests
import numpy as np  
import matplotlib.pyplot as plt  
#import seaborn as seabornInstance 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
%matplotlib inline

In [2]:
#Read dataset from URL 
url = 'https://raw.githubusercontent.com/mahmoudparsian/machine-learning-course/master/data/songs.csv'
dataset=pd.read_csv(url, encoding ='latin1')

In [3]:
#Preview the dataset
dataset.head()

,year,songtitle,artistname,songID,artistID,timesignature,timesignature_confidence,loudness,tempo,tempo_confidence,...,timbre_7_max,timbre_8_min,timbre_8_max,timbre_9_min,timbre_9_max,timbre_10_min,timbre_10_max,timbre_11_min,timbre_11_max,Top10
0,2010,This Is the House That Doubt Built,A Day to Remember,SOBGGAB12C5664F054,AROBSHL1187B9AFB01,3,0.853,-4.262,91.525,0.953,...,82.475,-52.025,39.116,-35.368,71.642,-126.440,18.658,-44.770,25.989,0
1,2010,Sticks & Bricks,A Day to Remember,SOPAQHU1315CD47F31,AROBSHL1187B9AFB01,4,1.000,-4.051,140.048,0.921,...,106.918,-61.320,35.378,-81.928,74.574,-103.808,121.935,-38.892,22.513,0
2,2010,All I Want,A Day to Remember,SOOIZOU1376E7C6386,AROBSHL1187B9AFB01,4,1.000,-3.571,160.512,0.489,...,80.621,-59.773,45.979,-46.293,59.904,-108.313,33.300,-43.733,25.744,0
3,2010,It's Complicated,A Day to Remember,SODRYWD1315CD49DBE,AROBSHL1187B9AFB01,4,1.000,-3.815,97.525,0.794,...,96.675,-78.660,41.088,-49.194,95.440,-102.676,46.422,-59.439,37.082,0
4,2010,2nd Sucks,A Day to Remember,SOICMQB1315CD46EE3,AROBSHL1187B9AFB01,4,0.788,-4.707,140.053,0.286,...,110.332,-56.450,37.555,-48.588,67.570,-52.796,22.888,-50.414,32.758,0


In [4]:
#Get basic information about dataset, as following showed there are not null values, not need to fill null value
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7574 entries, 0 to 7573
Data columns (total 39 columns):
year                        7574 non-null int64
songtitle                   7574 non-null object
artistname                  7574 non-null object
songID                      7574 non-null object
artistID                    7574 non-null object
timesignature               7574 non-null int64
timesignature_confidence    7574 non-null float64
loudness                    7574 non-null float64
tempo                       7574 non-null float64
tempo_confidence            7574 non-null float64
key                         7574 non-null int64
key_confidence              7574 non-null float64
energy                      7574 non-null float64
pitch                       7574 non-null float64
timbre_0_min                7574 non-null float64
timbre_0_max                7574 non-null float64
timbre_1_min                7574 non-null float64
timbre_1_max                7574 non-null float64
tim

In [5]:
dataset.shape

(7574, 39)

In [6]:
#See the statistical detail of this dataser
dataset.describe()

,year,timesignature,timesignature_confidence,loudness,tempo,tempo_confidence,key,key_confidence,energy,pitch,...,timbre_7_max,timbre_8_min,timbre_8_max,timbre_9_min,timbre_9_max,timbre_10_min,timbre_10_max,timbre_11_min,timbre_11_max,Top10
count,7574.000000,7574.000000,7574.000000,7574.000000,7574.000000,7574.000000,7574.000000,7574.000000,7574.000000,7574.000000,...,7574.000000,7574.000000,7574.000000,7574.000000,7574.000000,7574.000000,7574.000000,7574.000000,7574.000000,7574.000000
mean,2001.455902,3.893979,0.853307,-8.817262,107.348267,0.622866,5.384605,0.433843,0.675471,0.010817,...,95.653221,-63.703984,50.057489,-59.515016,68.028067,-87.340055,55.520722,-50.868457,47.490774,0.147742
std,5.815223,0.532561,0.243748,4.377079,24.665030,0.304656,3.572579,0.273826,0.243473,0.013671,...,26.859726,15.970244,14.463524,17.190527,20.381321,30.464274,23.549473,11.701530,12.463966,0.354868
min,1990.000000,0.000000,0.000000,-42.451000,0.000000,0.000000,0.000000,0.000000,0.000020,0.000000,...,15.699000,-158.756000,-25.955000,-149.507000,8.415000,-208.819000,-6.359000,-145.599000,7.200000,0.000000
25%,1997.000000,4.000000,0.819250,-10.847000,88.860250,0.372000,2.000000,0.204000,0.500138,0.003000,...,76.498000,-73.050750,40.584500,-70.282500,53.037250,-105.130250,39.196000,-58.058000,38.975250,0.000000
50%,2002.000000,4.000000,0.979000,-7.649000,103.268000,0.701500,6.000000,0.451500,0.718160,0.007000,...,94.632500,-62.661500,49.220000,-58.650000,65.935000,-83.074000,50.895000,-50.892500,46.437500,0.000000
75%,2006.000000,4.000000,1.000000,-5.640000,124.800500,0.892000,9.000000,0.646000,0.887396,0.014000,...,112.708250,-52.982750,58.462000,-47.699750,81.267000,-64.518000,66.593000,-43.292250,55.030250,0.000000
max,2010.000000,7.000000,1.000000,1.305000,244.307000,1.000000,11.000000,1.000000,0.998492,0.541000,...,214.819000,-2.382000,144.985000,1.140000,161.518000,-10.640000,192.417000,-6.497000,110.272000,1.000000


# 1.1 - Understanding the Data

### 1.1 How many observations (songs) are there in total?

In [7]:
len(dataset)

7574

### 1.2 How many Top-10?

In [8]:
Top_10_num = len(dataset[dataset['Top10'] == 1])
Top_10_num

1119

### 1.3 How many non Top-10?

In [9]:
non_Top_10_num = len(dataset[dataset['Top10'] == 0])
non_Top_10_num

6455

In [10]:
#Double check the sum is same with previous result
total_num=non_Top_10_num+Top_10_num
total_num

7574

# 1.2 - Understanding the Data

### How many songs does the dataset include for which the artist name is "Michael Jackson"?

In [11]:
MJ_num = len(dataset[dataset['artistname']== "Michael Jackson"])
MJ_num

18

# 2.1 - Creating Prediction Model

In [12]:
trainset=dataset[dataset['year']<=2009]
testset=dataset[dataset['year']>=2010]

In [13]:
#This showed train data and test data including all data
len(trainset)+len(testset)

7574

### How many observations (songs) are in the training set?

In [14]:
train_num = len(trainset)
train_num

7201

### How many observations (songs) are in the test set?

In [15]:
test_num = len(testset)
test_num

373

# 2.2 - Creating  Prediction Model

### Step 1: we want to exclude some of the variables in our dataset from being used as independent variables ("year", "songtitle", "artistname", "songID", and "artistID").

In [16]:
#Drop columns for train data sets
X_train = trainset.drop(["year", "songtitle", "artistname", "songID", "artistID", "Top10"], axis=1)
y_train = trainset.Top10

In [17]:
#Drop columns for test data sets
X_test = testset.drop(["year", "songtitle", "artistname", "songID", "artistID", "Top10"], axis=1)
y_test = testset.Top10

### Step 2: build a logistic regression model to predict Top10 using the training data.  You may enumerate all the remaining independent variables

In [18]:
lr=LogisticRegression()

In [19]:
lr.fit(X_train,y_train)

//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

# 2.3 - Creating Prediction Model

### Let's now think about the variables in our dataset related to the confidence of the time signature, key, and tempo (timesignature_confidence, key_confidence, and tempo_confidence). Our model seems to indicate that these confidence variables are significant (rather than the variables timesignature, key, and tempo themselves). What does the model suggest?

### A. The lower our confidence about time signature, key and tempo, the more likely the song is to be in the Top 10
### B. The higher our confidence about time signature, key and tempo, the more likely the song is to be in the Top 10

In [20]:
#Find the corralation between selected columns with "Top10" column
corr_ts_confi= dataset.corr()['Top10'][['timesignature_confidence']]
corr_ts_confi

timesignature_confidence    0.060802
Name: Top10, dtype: float64

In [21]:
corr_key_confi= dataset.corr()['Top10'][['key_confidence']]
corr_key_confi

key_confidence    0.010182
Name: Top10, dtype: float64

In [22]:
corr_tempo_confi= dataset.corr()['Top10'][['tempo_confidence']]
corr_tempo_confi

tempo_confidence    0.084852
Name: Top10, dtype: float64

### Answer B: All corralation for selected variables are positive. Therefore the answer B is right

# 2.4 - Creating Prediction Model

In [23]:
X_train.columns

Index(['timesignature', 'timesignature_confidence', 'loudness', 'tempo',
       'tempo_confidence', 'key', 'key_confidence', 'energy', 'pitch',
       'timbre_0_min', 'timbre_0_max', 'timbre_1_min', 'timbre_1_max',
       'timbre_2_min', 'timbre_2_max', 'timbre_3_min', 'timbre_3_max',
       'timbre_4_min', 'timbre_4_max', 'timbre_5_min', 'timbre_5_max',
       'timbre_6_min', 'timbre_6_max', 'timbre_7_min', 'timbre_7_max',
       'timbre_8_min', 'timbre_8_max', 'timbre_9_min', 'timbre_9_max',
       'timbre_10_min', 'timbre_10_max', 'timbre_11_min', 'timbre_11_max'],
      dtype='object')

In [24]:
# Paired coefficients with each variables
list(zip(X_train.columns, lr.coef_[0]))

[('timesignature', 0.25595736634635335),
 ('timesignature_confidence', 0.7502127263537941),
 ('loudness', 0.11908467924126351),
 ('tempo', 0.001898513381516748),
 ('tempo_confidence', 0.5729363013741708),
 ('key', 0.018833284323349568),
 ('key_confidence', 0.3412227074530119),
 ('energy', -1.1446252155378969),
 ('pitch', -0.22625889264973217),
 ('timbre_0_min', 0.025082613006561852),
 ('timbre_0_max', -0.1556494110831258),
 ('timbre_1_min', 0.006032227431534334),
 ('timbre_1_max', -0.0005370469864139916),
 ('timbre_2_min', -0.003282103479272596),
 ('timbre_2_max', 0.0002521927157005098),
 ('timbre_3_min', 0.0007616279260705753),
 ('timbre_3_max', -0.003009047259477765),
 ('timbre_4_min', 0.00789093357071326),
 ('timbre_4_max', 0.007139303291512327),
 ('timbre_5_min', -0.006513943789397778),
 ('timbre_5_max', 0.00036575738232949524),
 ('timbre_6_min', -0.01726698193653194),
 ('timbre_6_max', 0.005274577933273473),
 ('timbre_7_min', -0.005371005837465241),
 ('timbre_7_max', -0.0029732170

### Answer B: The coefficents of timesignature_confidence, key_confidence, and tempo_confidence are all positive. The high condifence might leads more likely to be Top10. Therefore less complex songs will have high possible to be Top10.

# 2.5 - Creating Prediction Model

### Answer A: The coefficents of loudness is positive. The heavy instrumentation might leads more likely to be Top10. 

# 3.1 - Validating Model

### Make predictions on the test set using our model. What is the accuracy of our model on the test set (Compute the accuracy as a number between 0 and 1.)

In [25]:
accurcy = lr.score(X_test, y_test)
accurcy

0.8632707774798928

# 3.2 - Validating Model

### What is the True Positive Rate of our model on the test set?

In [26]:
predictions = lr.predict(X_test)
cm = metrics.confusion_matrix(y_test, predictions)
print(cm)

[[313   1]
 [ 50   9]]


In [27]:
#print(classification_report(predictions, y_pred))

In [28]:
#tp / (tp + fn) 
TPR = metrics.recall_score(y_test, predictions)
TPR

0.15254237288135594

### What is the False Positive Rate of our model on the test set?

In [29]:
#fp / (fp + tn)
FPR=1/(313+1)
FPR

0.0031847133757961785